In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [61]:
df = pd.read_csv("C:\\Users\\LAU\\Downloads\\new proj\\base_data.csv",encoding="ISO-8859-1")

In [62]:
df.dropna(subset=['CustomerID'],how='all',inplace=True)


In [63]:
df['InvoiceDate'].max()

'9/30/2011 9:45'

In [64]:
import datetime as dt
now = dt.date(2011,12,9)

In [65]:
df['date'] = pd.DatetimeIndex(df.InvoiceDate).date

In [66]:
df.head()

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom,2010-12-01
1,536365,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01
2,536365,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom,2010-12-01
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01


In [67]:
recency_df = df.groupby(['CustomerID'],as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurchaseDate']
recency_df.head()

,CustomerID,LastPurchaseDate
0,12346.0,2011-01-18
1,12347.0,2011-12-07
2,12348.0,2011-09-25
3,12349.0,2011-11-21
4,12350.0,2011-02-02


In [68]:
df['Recency'] = recency_df.LastPurchaseDate.apply(lambda x : (now - x).days)

In [69]:
df.head()

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,Recency
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom,2010-12-01,325.0
1,536365,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,2.0
2,536365,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom,2010-12-01,75.0
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,18.0
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,310.0


In [70]:
recency_df.drop(columns=['LastPurchaseDate'],inplace=True)

In [71]:
df['Frequency'] = df.groupby('CustomerID')['InvoiceNo'].transform('nunique')



In [72]:
df.head()

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,Recency,Frequency
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom,2010-12-01,325.0,35
1,536365,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,2.0,35
2,536365,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom,2010-12-01,75.0,35
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,18.0,35
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,310.0,35


In [74]:
df['Total_cost'] = df['UnitPrice'] * df['Quantity']

In [75]:
df['Monetary'] = df.groupby('CustomerID')['Total_cost'].transform('sum')

In [76]:
df.head()

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,Recency,Frequency,Total_cost,Monetary
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom,2010-12-01,325.0,35,15.30,5288.63
1,536365,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,2.0,35,20.34,5288.63
2,536365,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom,2010-12-01,75.0,35,22.00,5288.63
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,18.0,35,20.34,5288.63
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,310.0,35,20.34,5288.63


In [77]:
mean_monetary = df[df['Monetary'] != 0]['Monetary'].mean()
df['Monetary'] = df['Monetary'].replace(0, mean_monetary)
df['Monetary'] = df['Monetary'].round().astype(int)

In [78]:
non_finite_values = df['Recency'].isna() | ~df['Recency'].isin([np.inf, -np.inf])
print(non_finite_values.any())
# For example, if you want to replace non-finite values with the mean of non-zero Recency values
mean_recency = df[df['Recency'] != 0]['Recency'].mean()
df['Recency'] = df['Recency'].replace([np.inf, -np.inf, np.nan], mean_recency)
# For example, if you want to replace non-finite values with the mean of non-zero Recency values
mean_recency = df[df['Recency'] != 0]['Recency'].mean()
df['Recency'] = df['Recency'].replace([np.inf, -np.inf, np.nan], mean_recency)



True


In [79]:
df['Recency'] = df['Recency'].round().astype(int)


In [80]:
df.head()

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,Recency,Frequency,Total_cost,Monetary
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom,2010-12-01,325,35,15.30,5289
1,536365,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,2,35,20.34,5289
2,536365,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom,2010-12-01,75,35,22.00,5289
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,18,35,20.34,5289
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,2010-12-01,310,35,20.34,5289


In [81]:
df["receny_score"] = pd.cut(df["Recency"].rank(method="first"), 5, labels=[1,2,3,4,5], duplicates='drop')
df["frequency_score"] = pd.cut(df["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4,5], duplicates='drop')
df["monetary_score"] = pd.cut(df["Monetary"].rank(method="first"),5,labels=[1, 2, 3, 4,5], duplicates='drop')

In [89]:
df['customer_segment'] = 1


# Example 2: Promising Customers (high Frequency, high Recency)-
promising_condition = (df['frequency_score'] >= 4) & (df['receny_score'] >= 4)
df.loc[promising_condition, 'customer_segment'] = 2

# Example 3: New Customers (low Frequency, high Recency)-
new_customer_condition = (df['frequency_score'] <= 2) & (df['receny_score'] >= 4)
df.loc[new_customer_condition, 'customer_segment'] = 3

# Example 4: Need Attention (low Frequency, low Recency)-
need_attention_condition = (df['frequency_score'] <= 2) & (df['receny_score'] <= 2)
df.loc[need_attention_condition, 'customer_segment'] = 4

# Example 5: At Risk (high Frequency, low Recency)-
at_risk_condition = (df['frequency_score'] >= 4) & (df['receny_score'] <= 2)
df.loc[at_risk_condition, 'customer_segment'] = 5

# Print the counts of each customer segment
print(df['customer_segment'].value_counts())

customer_segment
1    147489
3     70583
5     67028
4     62217
2     59512
Name: count, dtype: int64


In [90]:
df.head()

,InvoiceNo,Description,Quantity,UnitPrice,CustomerID,Country,date,Recency,Frequency,Total_cost,Monetary,receny_score,frequency_score,monetary_score,customer_segment
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,325,35,15.30,5289,5,5,4,2
1,536365,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,2,35,20.34,5289,1,5,4,5
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,75,35,22.00,5289,1,5,4,5
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,18,35,20.34,5289,1,5,4,5
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,310,35,20.34,5289,5,5,4,2


In [91]:
import os
print(os.getcwd())


c:\Users\LAU\Downloads\new proj


In [94]:
df.head()

,InvoiceNo,Description,Quantity,UnitPrice,CustomerID,Country,date,Recency,Frequency,Total_cost,Monetary,receny_score,frequency_score,monetary_score,customer_segment
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,325,35,15.30,5289,5,5,4,2
1,536365,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,2,35,20.34,5289,1,5,4,5
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,75,35,22.00,5289,1,5,4,5
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,18,35,20.34,5289,1,5,4,5
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,310,35,20.34,5289,5,5,4,2


In [96]:
df.to_excel('base_data_new_1.xlsx', index=False)